In [1]:
import os
import pandas as pd
import numpy as np
import geocoder
import feather
import re
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from tqdm import tqdm_notebook as tqdm

In [2]:
FOLDS_PATH = "../input/mynavi_Stratifiedfold_same_build.feather"
train = feather.read_dataframe(FOLDS_PATH)
n_train = len(train)
target = train["賃料"]
test = pd.read_csv('../input/test.csv')

train_length = train.shape[0]
test_length = test.shape[0]
all_df = pd.concat([train, test], axis=0, ignore_index=True)


access = feather.read_dataframe("../code/feature_csv/access_feature.feather")
all_df = pd.concat([all_df, access], axis=1)
base = feather.read_dataframe("../code/feature_csv/base_feature1.feather")
all_df = pd.concat([all_df, base], axis=1)

org_columns = all_df.columns

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [3]:
def modify_miss(df_input):
    df_input.loc[df_input["間取り"].str.contains("11R"), "間取り"] = "1R"
    
    df_input.loc[df_input["築年数"].str.contains("520年5ヶ月"), "築年数"] = "52年5ヶ月"
    
    df_input.loc[df_input["築年数"].str.contains("1019年7ヶ月"), "築年数"] = "19年7ヶ月"
    
    df_input.loc[df_input["面積"].str.contains("430.1m2"), "面積"] = "43.01m2"

    df_input.loc[df_input["面積"].str.contains("1m2"), "面積"] = "10m2"
    df_input.loc[df_input["面積"].str.contains("5.83m2"), "面積"] = "58.3m2"
    "三田線 三田(兵庫県)駅 徒歩14分"
    
    return df_input

def modify_rent(target, df_input):
    """ id == 5776 で賃料の0が一つ多い"""
    target = np.where(df_input["id"]==5776, 120350, target)
    
    return target

all_df = modify_miss(all_df)

all_df["賃料"] = modify_rent(all_df["賃料"], all_df)

all_df["same_build"] = all_df["building_floor"].astype(str) + all_df["地域_n丁目"] + all_df["int築年"].astype(str)

In [4]:
org_columns.values

array(['fold_id', 'id', 'same_build', 'アクセス', 'キッチン', 'バス・トイレ', '周辺環境',
       '契約期間', '室内設備', '建物構造', '所在地', '所在階', '放送・通信', '方角', '築年数', '賃料',
       '間取り', '面積', '駐車場', '路線_0', '駅_0', '駅からの距離_0', '路線_1', '駅_1',
       '駅からの距離_1', '路線_2', '駅_2', '駅からの距離_2', '駅_0からの距離_mean',
       '駅_0からの距離_std', '駅_0からの距離_min', '駅_0からの距離_max', '駅_1からの距離_mean',
       '駅_1からの距離_std', '駅_1からの距離_min', '駅_1からの距離_max', '駅_2からの距離_mean',
       '駅_2からの距離_std', '駅_2からの距離_min', '駅_2からの距離_max', '路線_0_counts',
       '路線_1_counts', '路線_2_counts', '駅_0_counts', '駅_1_counts',
       '駅_2_counts', '路線_counts_max', '路線_counts_mean', '路線_counts_min',
       '路線_counts_std', '駅_counts_max', '駅_counts_mean', '駅_counts_min',
       '駅_counts_std', '距離_max', '距離_min', '距離_mean', '距離_std',
       '路線_0_counts/距離', '路線_1_counts/距離', '路線_2_counts/距離',
       '駅_0_counts/距離', '駅_1_counts/距離', '駅_2_counts/距離',
       '路線counts/距離_max', '路線counts/距離_mean', '路線counts/距離_min',
       '路線counts/距離_std', '駅counts/距離_max', '駅coun

In [5]:
train = all_df[:n_train]
test = all_df[n_train:]

In [6]:
train["賃料per面積"] = train["賃料"] / train["float面積"]
test["賃料per面積"] = np.nan

#train_y = train["賃料"]
train_y = train["賃料per面積"]
folds = train["fold_id"]

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
target = "賃料per面積"    #所在_区 での面積　あたりの値段
columns = ["所在_区", "間取りtype", 'catuse_路線_0', 'catuse_駅_0', 'アクセス',
           "cat_int間取り", "cat_building_height", "地域名", '地域_n丁目', "same_build"
          ]
stat = "mean"
how_to_make_in_fold = "group"
result = []
n_split = 4

for column in tqdm(columns):
    data_tmp = pd.DataFrame({column:train[column], target:train[target]})
    target_mean = data_tmp.groupby(column)[target].mean()
    test[column] = test[column].map(target_mean)

for i in tqdm(range(folds.nunique())):
    tr_x, va_x = train[train["fold_id"]!=i].copy(), train[train["fold_id"]==i].copy()
    tr_y, va_y = train_y[train["fold_id"]!=i].copy(), train_y[train["fold_id"]==i].copy()
    
    tr_x = tr_x.reset_index(drop=True)
    tr_y = tr_y.reset_index(drop=True)
    tr_x["foldINfold_id"] = np.nan
    if how_to_make_in_fold == "str":
        skf_encoding = StratifiedKFold(n_splits=4, shuffle=True, random_state=100)
        for i, (train_index, valid_index) in enumerate(skf_encoding.split(tr_x["id"], tr_x["same_build"])):
            tr_x.loc[valid_index, "foldINfold_id"] = i
    
    if how_to_make_in_fold == "group":
        grp_encoding = GroupKFold(n_splits=4)
        for i, (train_index, valid_index) in enumerate(grp_encoding.split(tr_x["id"], tr_x["same_build"], tr_x["id"])):
            tr_x.loc[valid_index, "foldINfold_id"] = i
    
    
    for column in columns:
        data_tmp = pd.DataFrame({column:tr_x[column], target:tr_y, "foldINfold":tr_x["foldINfold_id"]})
        target_mean = data_tmp.groupby(column)[target].mean()
        va_x.loc[:, column] = va_x[column].map(target_mean)
        #va_x[column] = va_x[column].map(target_mean) #これあかんのん？

        tmp = np.repeat(np.nan, tr_x.shape[0])

        for i in range(n_split):
            target_mean = data_tmp[tr_x["foldINfold_id"]!=i].groupby(column)[target].mean()
            tmp[tr_x["foldINfold_id"]==i] = tr_x[column][tr_x["foldINfold_id"]==i].map(target_mean)

        tr_x.loc[:, column] = tmp
    
    tmp = pd.concat([tr_x, va_x])
    result.append(tmp)
    

# merge:ver 1min 1s
# map:ver    4.28 s

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior 

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
output_dir = '../code/feature_csv/'

""" target encoding 後のtest保存 """
test.to_csv(os.path.join(output_dir, f'target_encoding_test.csv'), index = False)

""""""
for i in tqdm(range(folds.nunique())):
    if how_to_make_in_fold == "group":
        result[i] = result[i].sort_values("id")
        result[i][columns + ["id", "fold_id"]].to_csv(os.path.join(output_dir, f'target_encoding_groupfold{i}.csv'), index = False)
    
    if how_to_make_in_fold == "str":
        result[i] = result[i].sort_values("id")
        result[i][columns + ["id", "fold_id"]].to_csv(os.path.join(output_dir, f'target_encoding_strfold{i}.csv'), index = False)
    #f'../code/feature_csv/target_encoding_fold{i}.csv'

In [9]:
result[0][columns].isnull().sum()

所在_区                       1
間取りtype                    0
catuse_路線_0                0
catuse_駅_0                 0
アクセス                   22570
cat_int間取り                 0
cat_building_height        0
地域名                       72
地域_n丁目                   900
same_build             15180
dtype: int64

In [13]:
#result[0].sort_values("id")
"""a = result[4]
v = a[a["fold_id"]==0]
t = a[a["fold_id"]!=0][columns + ["id"]]
a[a["fold_id"]!=4]"""
test["same_build"]

31470             NaN
31471     5427.587307
31472     2784.998143
31473             NaN
31474     1800.000000
             ...     
62727     3931.590623
62728     9925.000000
62729     3755.294854
62730    16466.774654
62731     3824.362606
Name: same_build, Length: 31262, dtype: float64